In [1]:
import numpy as np
import numpy.random as nr
import scipy.stats as ss
import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:


def vec_particle_gen(vp,nParticles):

    # Generate particles with maxwell distribution              
    # vp is the most probable speed of the maxwell distribution 


    velocity_vec = (vp/np.sqrt(2))*nr.normal( size = (3,1,nParticles) )
    
    phase_vec = nr.uniform(  size = (1,1,nParticles) )*2*np.pi
       
    amp_vec =  nr.normal( size = (3,1,nParticles) )  
    
    normfac = 1/np.sqrt(np.sum(amp_vec[:,0,:]**2,axis=0))
    
    amp_vec[:,0,:] =  amp_vec[:,0,:]*np.tile(normfac,(3,1))    

    
    return velocity_vec, amp_vec , phase_vec


def vector_random_walk(tX,omega,K,amp,phase):
    


    nParticles = omega.shape[2]
    nPoints = tX.shape[1]
    ret = np.ndarray( (3,nPoints ) )
    
    phase_tensor = np.tile(phase,(1,nPoints,1))
    amp_tensor = np.tile(amp,(1,nPoints,1))
                     
    for i in [0,1,2]:          
        ret[i,:] = np.sum( amp_tensor[i,:,:]*np.sin(np.outer(tX[0,:],omega[0,0,:]) 
                                  - np.matmul(tX[1:,:].transpose(),K[:,0,:])
                                  + phase_tensor[:,0,:]), axis=1) 
    return ret




Statistics.

In [ ]:
'''
Get the marginal distribution
'''
vp=1e-3
log10m = -22
m=10**log10m

t_corr = 2*np.pi/m
l_dB = 2*np.pi/m/vp
#ev2kpc = 3.0857e19 /1.973e-7


nPoints = 16

tx = np.zeros((4,nPoints))
tlist= np.linspace(0,1,nPoints+1)
tx[0,:] = tlist[:-1]*t_corr

nParticles = 2**14
output=[]

nSimulation= 10000
Amps = np.ndarray((3,nSimulation))
Phis = np.ndarray((3,nSimulation))


'''
Main:
'''
for i in tqdm(range(nSimulation)):
    v,a,p=vec_particle_gen(vp,nParticles)

    # Physical values   
    m = 10**log10m # eV
    K = m*v

    # From K to omega
    omega = np.ndarray((1,nPoints,nParticles))
    omega[0,:,:] = np.sqrt(np.sum(K*K,axis=0)+m**2)
    
    #Retx(3,nPoints)
    Ret = vector_random_walk(tx,omega,K,a,p)
    
    #results(3,nSimulation)
    Amps[:,i] = 2 * np.sum(Ret*Ret,axis=1)/nPoints #  mean value over time
    Phis[0,i] = tlist[np.where(np.max(Ret[0]) == Ret[0])[0][0]]*2*np.pi
    Phis[1,i] = tlist[np.where(np.max(Ret[1]) == Ret[1])[0][0]]*2*np.pi
    Phis[2,i] = tlist[np.where(np.max(Ret[2]) == Ret[2])[0][0]]*2*np.pi


In [ ]:
# Data processing
'''
Fit with gamma functions
'''
y=Amps[1,:]
y=y/np.mean(y)
y=np.sqrt(y)


plt.figure(figsize=(6,4))
# Histogram
Pr,bins,fig=plt.hist(y,30,density=True,label="Marginal Distribution of $A_{0}$",color="steelblue");
midbins = np.diff(bins)/2.+bins[:-1]

# curve fit
a,loc,scale = ss.gamma.fit(y,floc=0)
Pr_fit = ss.gamma.pdf(x=midbins,a=a,loc=loc,scale=scale)

plt.plot(midbins,Pr_fit,'black',linewidth=6,alpha=0.6,label='Gamma Fit')
plt.annotate(s="a = %.3f\n  scale = %.3f"%(a,scale),
             horizontalalignment="right",
         bbox=dict(boxstyle="round",fc="w"),xy=(0.98,0.5),xycoords='axes fraction',size=10)
plt.xlabel("Normalized $A_0$")
plt.ylabel("$Pr(A_0)$")
plt.legend()
#plt.savefig('GammaFit.pdf')

In [ ]:
# Check the linear combination of A_i
# Ak=np.sqrt((Ax**2+Ay**2+2*Ax*Ay*np.sin(Phix+Phiy))/2.)

In [3]:
'''
COHERENCE

Check the space coherence
'''
vp=1e-3
log10m = -22
m=10**log10m

t_corr = 2*np.pi/m
l_dB = 2*np.pi/m/vp
#ev2kpc = 3.0857e19 /1.973e-7


nPoints = 16

tx = np.zeros((4,nPoints))
tlist= np.linspace(0,1,nPoints+1)
tx[0,:] = tlist[:-1]*t_corr

ty = np.zeros((4,nPoints))
ty[0,:] = tlist[:-1]*t_corr
ty[1,:] = np.tile(l_dB,nPoints)*1.00

nParticles = 2**10
output=[]

nSimulation= 10000
Amps_x = np.ndarray((3,nSimulation))
Phis_x = np.ndarray((3,nSimulation))
Amps_y = np.ndarray((3,nSimulation))
Phis_y = np.ndarray((3,nSimulation))

'''
Main:
'''
for i in tqdm(range(nSimulation)):
    v,a,p=vec_particle_gen(vp,nParticles)

    # Physical values   
    m = 10**log10m # eV
    K = m*v

    # From K to omega
    omega = np.ndarray((1,1,nParticles))
    omega[0,:,:] = np.sqrt(np.sum(K*K,axis=0)+m**2)

    #Retx(3,nPoints)

    Ret_x = vector_random_walk(tx,omega,K,a,p)
    Ret_y = vector_random_walk(ty,omega,K,a,p)
    
    #results(3,nSimulation)
    Amps_x[:,i] = 2 * np.sum(Ret_x*Ret_x,axis=1)/nPoints #  mean value over time
    Phis_x[0,i] = tlist[np.where(np.max(Ret_x[0]) == Ret_x[0])[0][0]]*2*np.pi
    Phis_x[1,i] = tlist[np.where(np.max(Ret_x[1]) == Ret_x[1])[0][0]]*2*np.pi
    Phis_x[2,i] = tlist[np.where(np.max(Ret_x[2]) == Ret_x[2])[0][0]]*2*np.pi
    
    Amps_y[:,i] = 2 * np.sum(Ret_y*Ret_y,axis=1)/nPoints #  mean value over time
    Phis_y[0,i] = tlist[np.where(np.max(Ret_y[0]) == Ret_y[0])[0][0]]*2*np.pi
    Phis_y[1,i] = tlist[np.where(np.max(Ret_y[1]) == Ret_y[1])[0][0]]*2*np.pi
    Phis_y[2,i] = tlist[np.where(np.max(Ret_y[2]) == Ret_y[2])[0][0]]*2*np.pi


100%|██████████| 10000/10000 [00:25<00:00, 390.29it/s]


In [4]:
'''
Calculate the correlation !!!
'''

a = Amps_x[0,:]
b = Amps_y[0,:]
c = Amps_y[1,:]
d = Phis_x[0,:]
e = Phis_y[0,:]
f = Phis_y[1,:]
results=[]
for (x2,y2) in [(a,b),(a,c),(d,e),(d,f),(a,e),(a,f)]:

    x = np.sqrt(x2)
    y = np.sqrt(y2)

    m_x2 = np.mean(x2)
    m_x = np.mean(x)
    m_y2 = np.mean(y2)
    m_y = np.mean(y)

    sig_x = np.sqrt(m_x2-m_x**2)
    sig_y = np.sqrt(m_y2-m_y**2)

    results.append(np.mean((x-m_x)*(y-m_y))/sig_x/sig_y)

print(results)

[0.010715222280837794, 0.00030508900737771604, -0.011281326224930574, 0.00047804600866564815, -0.018573386406022734, 0.01660515939233207]


In [6]:
kpc2nat = 3.0857e19 /1.973e-7


l = 0.5*kpc2nat/(2*np.pi/10**(-21.5)/vp)
print("%.3f"%(l))

3.936


In [ ]:
corr = np.ndarray((10,6))
Dis=np.array([0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12])

corr[0,:] = np.array([0.9976607022205033, 0.0037676705818899463, 0.9327020080089024, 0.007450448389243849, -0.009171508248407718, 0.005356242440316161])
corr[1,:] = np.array([0.99106258203287, 0.011099269792954449, 0.9012317040062288, 0.004039380559944105, -0.0034537435583275934, 0.004679418272634157])
corr[2,:] = np.array([0.9643100923845139, -0.000992748126691237, 0.7965985256746666, -0.01105591066757477, 0.00970309745521556, 0.009073964597856511])
corr[3,:] = np.array([0.8623397830753567, 0.007114731531677067, 0.6580210721689032, 0.006986199182822402, -0.001957350351134233, 0.013300395224419708])
corr[4,:] = np.array([0.5721820929063567, -0.007412828990754051, 0.43119528453945477, -0.0036480322991355608, 0.005106476076536831, -0.003291443323211755])
corr[5,:] = np.array([0.11629739390099995, 0.0059398877230059325, 0.17160920059913792, -0.007730739043007247, -0.015587786942008451, 0.004431751607055087])
corr[6,:] = np.array([0.0004846887852723556, 0.00923196440467667, 0.008129350252005289, 0.002672610405310479, -0.014295219328776707, 0.014190959985873657])
corr[7,:] = np.array([0.007517423340686037, 0.020655454466178978, -0.0008678267229125785, 0.005190865100771142, -0.018050857571350856, 0.004259351720611055])
corr[8,:] = np.array([-0.009911551109561472, 0.019040773634095867, -0.004305765239621097, -0.008191116532864086, 0.00571505800291175, -0.021481308743130006])
corr[9,:] = np.array([0.00997750749582416, 0.012745851516836933, -0.008872693213043598, 0.0048421096593427556, 0.00752769696005557, 0.0009189869377229605])




perc235=np.array([0.039,0.118])
perc23=np.array([0.124,0.373])
perc225=np.array([0.393,1.181])
perc22=np.array([1.245,3.734])




In [ ]:
plt.figure(figsize=(6,4))
ll = -0.1
hl = 1.19
lel = 10**-1.8
ril = 10**0.7
plt.ylim([ll,hl])
plt.xlim([lel,ril])

plt.semilogx(Dis,corr[:,0],label="$A_{0,x},A_{0,x}$",linewidth=3);
plt.semilogx(Dis,corr[:,1],label="$A_{0,x},A_{0,y}$",linewidth=3);
plt.semilogx(Dis,corr[:,2],label="$\phi_{x},\phi_{x}$",linewidth=3);
plt.semilogx(Dis,corr[:,3],label="$\phi_{x},\phi_{y}$",linewidth=3);
plt.semilogx(Dis,corr[:,4],label="$A_{0,x},\phi_{x}$",linewidth=3);
plt.semilogx(Dis,corr[:,5],label="$A_{0,x},\phi_{y}$",linewidth=3);

plt.plot([lel,ril],[0,0],'k--')
plt.plot([lel,ril],[1,1],'k--')

plt.fill_between(perc235,[hl,hl],[ll,ll],facecolor='gold',alpha=0.2)
plt.fill_between(perc23,[hl,hl],[ll,ll],facecolor='gold',alpha=0.3)
plt.fill_between(perc225,[hl,hl],[ll,ll],facecolor='gold',alpha=0.4)
plt.fill_between(perc22,[hl,hl],[ll,ll],facecolor='gold',alpha=0.5)


plt.annotate("$m =$",horizontalalignment="right",xy=[perc235[0]*0.9,1.05],xycoords='data',size=15)
plt.annotate("$10^{-23.5}$eV",horizontalalignment="left",xy=[perc235[0],1.05],xycoords='data',size=12)
plt.annotate("$10^{-23}$eV",horizontalalignment="left",xy=[perc23[0],1.05],xycoords='data',size=12)
plt.annotate("$10^{-22.5}$eV",horizontalalignment="left",xy=[perc225[0],1.05],xycoords='data',size=12)
plt.annotate("$10^{-22}$eV",horizontalalignment="left",xy=[perc22[0],1.05],xycoords='data',size=12)



plt.xlabel("$|x-y|/\lambda_{dB}$")
plt.ylabel("$corr(X,Y)$")
plt.legend()
plt.savefig('SpatialCorr.pdf')

In [ ]:
# Correlation Test-1, on amplitudes if certain direction 



valsx=np.array([0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56])
y1=[0.9999404204236939, 0.9997286376075016, 1.000073408701364,
 0.9999029418075727, 0.9998593391841898, 0.9993517781991963,
 0.9999465206974555, 0.9997576026360162, 0.9997438530657218]
y2=[0.9410427166670868, 0.939193879567938, 0.8845677195484672,
 0.7030079033709721, 0.48367021627090495, 0.4436117219154799,
 0.5052972335342074, 0.4743389963528783, 0.4673111944432566]
y100=[0.9810734888042447, 0.9408280718582631, 0.7925041174433759,
 0.3779713953921857, 0.015635133909090475, -0.048319716596257015,
 -0.047782568367157295, -0.034522005224365514, -0.027387532520900416]
y1000=[1.0012748295717215, 0.9706890421176969, 0.8175871118817806,
 0.46207917938094806, 0.053702680841951074, 0.015464042416849157,
 -0.00047152232801450633, -0.015587436498420686, 0.021327256909953848]




plt.semilogx(valsx,y1,'--')
plt.semilogx(valsx,y2)
plt.semilogx(valsx,y100)
plt.semilogx(valsx,y1000)

In [ ]:
# Amplitude as the function on N 

sqrtN=np.array(range(100)) 
rmsA=np.array([ 0.        ,  0.79846634,  1.61470283,  2.45912006,  3.2960502 ,
        4.10531182,  5.00936223,  5.65628151,  6.3610982 ,  7.22432485,
        8.18869485,  9.03596424,  9.89567429, 10.64053398, 11.64746189,
       11.94587072, 13.09847378, 13.79092467, 14.75556847, 15.46454741,
       16.52603156, 16.41034475, 18.03788818, 18.90438091, 19.88455712,
       20.67653797, 21.15476708, 21.74219317, 23.02839755, 23.94460244,
       24.5135315 , 25.62289196, 26.29387735, 26.60596482, 28.03526087,
       29.06469686, 29.13845635, 30.22109512, 29.74143123, 32.27197029,
       32.82808096, 33.09821604, 34.68934578, 35.6393422 , 36.43995517,
       36.91823923, 36.90650459, 38.42189739, 39.40285861, 40.08102474,
       40.9896192 , 42.48226713, 43.43985562, 44.51905154, 42.98135117,
       44.77947688, 45.86301141, 46.98701022, 46.67341974, 47.0655024 ,
       48.6029759 , 49.74842101, 51.54711642, 51.17306504, 53.33317334,
       53.1395299 , 54.64647966, 55.35221691, 56.6030676 , 56.45643468,
       56.49861522, 58.41489342, 59.35357932, 59.36948378, 60.31340694,
       61.45030096, 62.39700416, 62.3806855 , 64.7663445 , 64.61164241,
       65.74092418, 65.14686267, 66.73730258, 68.55993408, 68.8997811 ,
       69.96596792, 69.41275499, 72.39012902, 73.38264649, 73.58243626,
       72.77080659, 74.78611553, 74.98845046, 76.61442859, 78.7100685 ,
       76.14114011, 78.34209638, 78.63969245, 80.9677009 , 81.48174609])
meanA=np.array([ 0.        ,  0.62075403,  1.28704153,  1.96499303,  2.63348202,
        3.26266941,  3.9939252 ,  4.51085347,  5.06498872,  5.76844449,
        6.50449084,  7.20356297,  7.95839795,  8.5058701 ,  9.2555877 ,
        9.55251364, 10.43833793, 11.14392604, 11.62711766, 12.32752206,
       13.20932638, 13.14560063, 14.38737839, 15.02930709, 15.83167965,
       16.60510042, 16.81611099, 17.3031717 , 18.42177316, 19.08763107,
       19.47287412, 20.53271363, 20.91497086, 21.31568612, 22.4999253 ,
       23.41616609, 23.40032242, 24.02719153, 23.76023575, 25.66935356,
       26.07803993, 26.23784541, 27.60667146, 28.36006271, 29.14530548,
       29.66343084, 29.53712944, 30.67012787, 31.2958451 , 31.72407652,
       32.66972443, 33.74169982, 34.85269115, 35.65371956, 34.16837476,
       36.01088549, 36.76882923, 37.67857615, 37.39372945, 37.309618  ,
       38.97601965, 39.7727888 , 41.24697693, 40.32327808, 42.32301946,
       42.6533967 , 43.80413696, 44.47956186, 45.02282014, 45.1215823 ,
       45.16834647, 46.57989769, 47.7396148 , 47.25986284, 48.26060286,
       48.96629721, 49.86263602, 49.72826816, 51.69158344, 51.95797653,
       52.38111997, 52.45743527, 53.37935914, 55.11065483, 54.93432823,
       56.02484397, 55.78532571, 58.41918657, 58.02068968, 58.82864473,
       57.94621609, 60.20022989, 59.7776205 , 61.23974151, 62.60424592,
       60.37561381, 62.63116057, 62.59372356, 64.98788157, 65.38171219])

plt.plot(sqrtN,rmsA)
plt.plot(sqrtN,meanA)

In [ ]:
'''
Setups, change the parameters here!
'''
vp=1e-3
log10m = -22
m=10**log10m

t_corr = 2*np.pi/m
l_dB = 2*np.pi/m/vp
#ev2kpc = 3.0857e19 /1.973e-7


nTime = 10
nSpace = 100
nPoints = nTime * nSpace

tz = np.zeros((4,nPoints))
tz[0,:] = np.repeat(np.linspace(0,1,nTime)*t_corr,nSpace)
tz[1,:]= np.tile(ss.uniform.rvs(size=nSpace),nTime)*l_dB*10
tz[2,:]= np.tile(ss.uniform.rvs(size=nSpace),nTime)*l_dB*10
tz[3,:]= np.tile(ss.uniform.rvs(size=nSpace),nTime)*l_dB*10

nParticles = 1000

nSimulation= 10000
A02 = np.ndarray((nSpace,3,nSimulation))


for i in tqdm(range(nSimulation)):
    v,a,p=vec_particle_gen(vp,nParticles)

    # Physical values   
    m = 10**log10m # eV
    K = m*v

    # From K to omega
    omega = np.ndarray((1,nParticles))
    omega[0,:] = np.sqrt(np.sum(K*K,axis=0)+m**2)

    #Retx(3,nPoints)

    Ret = vector_random_walk(tz,omega,K,a,p)

    #results(3,nSimulation)
    for k in range(nSpace):
        A02[k,:,i] = 2* np.sum(Ret[:,k::nSpace]*Ret[:,k::nSpace],axis=1)/nTime #  mean value over time




In [ ]:
Pr,bins,fig=plt.hist(np.sqrt(A02[100,0,:]),50);

d = bins[1]-bins[0]
Pr = Pr*(1./nSimulation)*(1./d)
midbins = np.diff(bins)/2+bins[:-1]


#plt.hist(np.sqrt(A02[:,0,10]),20);

#np.mean(A02[:,0,:])/2

In [ ]:
plt.plot(midbins,Pr)
plt.plot(midbins,ss.maxwell.pdf(midbins/12.)/12.)

In [ ]:
'''
Get the statistical distribution of A**2 on uniformly distributed time-space coordinate.

Also the Monte-Carlo approximation of the integral.
'''
vp=1e-3
log10m = -22
m=10**log10m

t_corr = 2*np.pi/m
l_dB = 2*np.pi/m/vp
#ev2kpc = 3.0857e19 /1.973e-7


nPoints = 1000

tz = ss.uniform.rvs(size=(4,nPoints))
tz[0,:]=tz[0,:]*t_corr
tz[1:,:]=tz[1:,:]*l_dB*10

nParticles = 1000


nSimulation= 1000
outputC=np.ndarray((3,nSimulation))

'''
This part is slow:
'''
for i in tqdm(range(nSimulation)):
    v,a,p=vec_particle_gen(vp,nParticles)

    # Physical values   
    m = 10**log10m # eV
    K = m*v

    # From K to omega
    omega = np.ndarray((1,nParticles))
    omega[0,:] = np.sqrt(np.sum(K*K,axis=0)+m**2)

    #Ret(3,nPoints)

    Ret = vector_random_walk(tz,omega,K,a,p)

    outputC[:,i]=np.sum(Ret*Ret,axis=1)/nPoints




In [ ]:
v,a,p=vec_particle_gen(vp,1000)
plt.hist(a[0,:])